In [4]:
import cv2
from glob import glob
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv3D,MaxPool3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [6]:
data = pathlib.Path('/content/drive/MyDrive/images/Train')
data

PosixPath('/content/drive/MyDrive/images/Train')

In [7]:
import glob
image=glob.glob(str(data)+'/*')
image

['/content/drive/MyDrive/images/Train/Athulmk',
 '/content/drive/MyDrive/images/Train/Akshay',
 '/content/drive/MyDrive/images/Train/Sreeshnu']

In [8]:
A=list(data.glob('Athulmk/*.jpg'))
B=list(data.glob('Akshay/*.jpg'))
C=list(data.glob('Sreeshnu/*.jpg'))

In [9]:
len(A),len(B),len(C)

(361, 513, 312)

In [10]:
data1={'Athulmk':A,
       'Akshay':B,
       'Sreeshnu':C}
data2={'Athulmk':0,
       'Akshay':1,
       'Sreeshnu':2}

In [11]:
data2

{'Athulmk': 0, 'Akshay': 1, 'Sreeshnu': 2}

In [12]:
x=[]
y=[]
for i in data1:
  data_name=i
  data_list=data1[data_name]
  for path in data_list:
    img=cv2.imread(str(path))
    img=cv2.resize(img,(224,224))
    img=img/255
    x.append(img)
    cls=data2[data_name]
    y.append(cls)

In [13]:
len(x)

1186

In [14]:
x=np.array(x)
y=np.array(y)

In [15]:
y

array([0, 0, 0, ..., 2, 2, 2])

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.75,random_state=1)

In [18]:
len(x_train),len(x_test),len(y_train),len(y_test)

(889, 297, 889, 297)

In [19]:
x_train.shape,x_test.shape

((889, 224, 224, 3), (297, 224, 224, 3))

In [20]:
y_train.shape,y_test.shape

((889,), (297,))

In [21]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [22]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                             include_top=False,
                                             weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [23]:
print(base_model.summary())

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [24]:
pip install keras.layers.core

ERROR: Could not find a version that satisfies the requirement keras.layers.core (from versions: none)
ERROR: No matching distribution found for keras.layers.core


In [25]:
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.models import Model

headmodel = base_model.output
headmodel = MaxPooling2D(pool_size=(2, 2))(headmodel)
headmodel = Flatten(name='flatten')(headmodel)

headmodel = Dense(32, activation='relu')(headmodel)
headmodel = Dropout(0.2)(headmodel)
headmodel = Dense(30, activation='softmax')(headmodel)

model = Model(inputs=base_model.input, outputs=headmodel)

for layer in base_model.layers:
  layer.trainable = False

In [26]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=1e-4)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model_hist = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test), batch_size=30)

Epoch 1/15
30/30 [==============================] - 16s 282ms/step - loss: 0.6752 - accuracy: 0.8324 - val_loss: 0.0029 - val_accuracy: 1.0000
Epoch 2/15
30/30 [==============================] - 2s 72ms/step - loss: 0.1006 - accuracy: 0.9640 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/15
30/30 [==============================] - 2s 76ms/step - loss: 0.0689 - accuracy: 0.9809 - val_loss: 5.9958e-04 - val_accuracy: 1.0000
Epoch 4/15
30/30 [==============================] - 2s 70ms/step - loss: 0.0648 - accuracy: 0.9786 - val_loss: 3.6698e-04 - val_accuracy: 1.0000
Epoch 5/15
30/30 [==============================] - 2s 61ms/step - loss: 0.0470 - accuracy: 0.9899 - val_loss: 1.8355e-04 - val_accuracy: 1.0000
Epoch 6/15
30/30 [==============================] - 2s 70ms/step - loss: 0.0428 - accuracy: 0.9888 - val_loss: 1.4178e-04 - val_accuracy: 1.0000
Epoch 7/15
30/30 [==============================] - 2s 77ms/step - loss: 0.0509 - accuracy: 0.9854 - val_loss: 7.6649e-05 - val_accuracy

In [27]:
from tensorflow.keras.preprocessing import image
def testingimage(image_directory):
  test_image=image.load_img(image_directory,target_size=(224,224))
  test_image=image.img_to_array(test_image)
  test_image=np.expand_dims(test_image,axis=0)
  test_image=test_image/255
  result=model.predict(x=test_image)
  print(result)
  if np.argmax(result)==0:
    prediction='Athul M K'
  elif np.argmax(result)==1:
    prediction='Akshay M K'
  elif np.argmax(result)==2:
    prediction='Sreeshnu K'
  else:
    prediction='unknown'
  return prediction


In [28]:
print(testingimage(r"/content/drive/MyDrive/Athul M K NEW.jpg"))




1/1 [==============================] - 1s 1s/step
[[2.8105384e-05 9.9069947e-01 6.0681934e-03 1.3475178e-10 1.8287762e-08
  3.7685806e-11 8.5928926e-09 4.8508144e-09 4.6842550e-07 2.2840393e-14
  1.7028931e-04 7.4453671e-10 2.8625088e-08 1.2524125e-10 6.8634006e-07
  4.0550674e-11 2.5515820e-07 1.0812214e-10 5.8835754e-09 3.4803594e-05
  2.6872286e-07 1.6458666e-06 3.7734327e-09 2.9108051e-08 6.2797412e-06
  1.3544428e-12 3.8412679e-11 2.9857033e-03 3.6492381e-06 2.7914071e-09]]
Akshay M K


In [29]:
model.save('New Image.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
